<a href="https://colab.research.google.com/github/SaqlainHussainShah/PyOD/blob/master/Copy_of_LOCI_PYOD_5_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install loci
!pip install pyod

     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 2.1MB 8.7MB/s 
  Created wheel for pyod: filename=pyod-0.8.1-cp36-none-any.whl size=105653 sha256=c271ee3b3458c4ad31b2fb6e5008119c375479fb82e1813068c62473bca5c486
  Stored in directory: /root/.cache/pip/wheels/2e/ca/18/727e9d98a41f5f4385a97d5b429f3a9c8fbee13f9780c18642
  Created wheel for combo: filename=combo-0.1.1-cp36-none-any.whl size=42111 sha256=9fb6076f11799876d64c278763187f1b9313a9a0865f49d934b52efc62c6f0c3
  Stored in directory: /root/.cache/pip/wheels/55/ec/e5/a2331372c676c467e70c6646e646edf6997d5c4905b8c0f5e6
  Created wheel for suod: filename=suod-0.0.4-cp36-none-any.whl size=2167157 sha256=a21e222c4701bd3f587f742b8a26d53520e588ded8e1039dd85734479e78907f
  Stored in directory: /root/.cache/pip/wheels/57/55/e5/a4fca65bba231f6d0115059b589148774b41faea25b3f2aa27
Successfully built pyod combo suod


In [2]:
#Importing Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split     # import module for train test split
from pyod.models.knn import KNN
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.loci import LOCI
from pyod.utils.data import evaluate_print

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
df1=pd.read_csv("/content/drive/My Drive/CICIDS/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/content/drive/My Drive/CICIDS/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
#df3=pd.read_csv("../Datasets/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
#df4=pd.read_csv("../Datasets/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
#df5=pd.read_csv("../Datasets/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
#df6=pd.read_csv("../Datasets/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
#df7=pd.read_csv("../Datasets/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
#df8=pd.read_csv("../Datasets/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")


In [5]:

df=pd.concat([df1,df2])
#df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [6]:
del df1,df2
#del df1,df2,df3,df4,df5,df6,df7,df8


In [7]:
# category A attacks
df_bot=df[df[' Label']=='Bot']
df_ddos=df[df[' Label']=='DDoS']
df_golden_eye=df[df[' Label']=='DoS GoldenEye']
df_hulk=df[df[' Label']=='DoS Hulk']
df_ssh_patator=df[df[' Label']=='SSH-Patator']
df_ftp_patator=df[df[' Label']=='FTP-Patator']
df_infiltration=df[df[' Label']=='Infiltration']

In [8]:
# benign data
df_benign=df[df[' Label']=='BENIGN']

In [9]:
df_cat_A=pd.concat([df_bot,df_ddos,df_golden_eye,df_hulk,df_ssh_patator,df_ftp_patator,df_infiltration, df_benign])
df_cat_A.groupby(' Label').first()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,54865,3,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.00000,4.000000e+06,666666.666700,3.0000,0.0000,3,3,3,3.0,0.000000,3,3,0,0.0000,0.0000,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,0,0,0,0,9.0,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
DDoS,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,5840,0,1658.142857,2137.29708,8.991399e+03,7.729218,143754.6667,430865.8067,1292730,2,747,373.5,523.966125,744,3,1293746,215624.3333,527671.9348,1292730,2,0,0,0,0,72,152,2.318765,5.410452,0,5840,1057.545455,1853.437529,3435230.673,0,0,0,1,0,0,0,0,2,1163.3,8.666667,1658.142857,72,0,0,0,0,0,0,3,26,7,11607,8192,229,2,20,0.0,0.0,0,0,0.0,0.0,0,0


In [10]:
del df_bot,df_ddos,df_golden_eye,df_hulk,df_ssh_patator,df_ftp_patator,df_infiltration, df_benign

In [11]:
df=df_cat_A.copy()

In [12]:
del df_cat_A

In [13]:
df.groupby(' Label').first()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,54865,3,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.00000,4.000000e+06,666666.666700,3.0000,0.0000,3,3,3,3.0,0.000000,3,3,0,0.0000,0.0000,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,0,0,0,0,9.0,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
DDoS,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,5840,0,1658.142857,2137.29708,8.991399e+03,7.729218,143754.6667,430865.8067,1292730,2,747,373.5,523.966125,744,3,1293746,215624.3333,527671.9348,1292730,2,0,0,0,0,72,152,2.318765,5.410452,0,5840,1057.545455,1853.437529,3435230.673,0,0,0,1,0,0,0,0,2,1163.3,8.666667,1658.142857,72,0,0,0,0,0,0,3,26,7,11607,8192,229,2,20,0.0,0.0,0,0,0.0,0.0,0,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353282 entries, 18883 to 286466
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             353282 non-null  int64  
 1    Flow Duration                353282 non-null  int64  
 2    Total Fwd Packets            353282 non-null  int64  
 3    Total Backward Packets       353282 non-null  int64  
 4   Total Length of Fwd Packets   353282 non-null  int64  
 5    Total Length of Bwd Packets  353282 non-null  int64  
 6    Fwd Packet Length Max        353282 non-null  int64  
 7    Fwd Packet Length Min        353282 non-null  int64  
 8    Fwd Packet Length Mean       353282 non-null  float64
 9    Fwd Packet Length Std        353282 non-null  float64
 10  Bwd Packet Length Max         353282 non-null  int64  
 11   Bwd Packet Length Min        353282 non-null  int64  
 12   Bwd Packet Length Mean       353282 non

In [15]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,353282.000000,3.532820e+05,353282.000000,353282.000000,353282.000000,3.532820e+05,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,3.532630e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,353282.000000,353282.0,353282.0,353282.0,353282.000000,353282.000000,3.532820e+05,3.532820e+05,353282.000000,353282.000000,353282.000000,353282.000000,3.532820e+05,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.0,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.000000,353282.0,353282.0,353282.0,353282.0,353282.0,353282.0,353282.000000,353282.000000,353282.000000,3.532820e+05,353282.000000,353282.000000,353282.000000,353282.000000,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05
mean,8317.536203,1.470301e+07,5.473658,5.324557,789.085334,5.998437e+03,409.347516,25.766543,124.442551,157.246116,1894.606150,30.283836,627.872996,830.735548,1.180069e+06,3.142177e+04,1.276702e+06,3.236764e+06,1.006059e+07,2.726960e+04,1.406242e+07,2.220016e+06,3.731268e+06,9.705286e+06,4.615816e+05,8.187502e+06,1.216883e+06,1.349703e+06,4.155637e+06,5.192539e+05,0.038253,0.0,0.0,0.0,131.086803,129.805991,2.754550e+04,4.078146e+03,12.924163,2225.132843,366.434038,742.017821,1.810766e+06,0.008755,0.038253,0.000136,0.310455,0.422818,0.126151,0.0,0.000136,0.885777,409.788876,124.442551,627.872996,131.086803,0.0,0.0,0.0,0.0,0.0,0.0,5.473658,789.085334,5.324557,5.998437e+03,4875.271791,1240.889742,3.515919,22.886063,1.457166e+05,2.687304e+04,1.960644e+05,1.310762e+05,7.859771e+06,2.357610e+06,9.528822e+06,6.165548e+06
std,18974.050876,3.159391e+07,21.336977,30.756145,3091.009774,5.552206e+04,1526.654248,132.061381,412.573844,650.176230,3203.740312,62.200662,975.733774,1493.026770,2.676003e+07,1.775484e+05,2.851236e+06,7.048068e+06,2.317754e+07,6.970328e+05,3.158060e+07,6.990682e+06,9.124326e+06,2.372194e+07,5.824483e+06,2.555951e+07,6.506502e+06,4.866280e+06,1.498481e+07,6.130743e+06,0.191806,0.0,0.0,0.0,541.233087,747.721449,1.645333e+05,3.849911e+04,20.033593,3368.488774,498.562559,1122.684838,3.541153e+06,0.093158,0.191806,0.011656,0.462680,0.494008,0.332020,0.0,0.011656,1.214095,556.123626,412.573844,975.733774,541.233087,0.0,0.0,0.0,0.0,0.0,0.0,21.336977,3091.009774,30.756145,5.552206e+04,10018.412742,6722.190948,16.529588,5.380668,7.644236e+05,2.895786e+05,9.742138e+05,7.354875e+05,1.929626e+07,1.039089e+07,2.328394e+07,1.762792e+07
min,0.000000,-

In [16]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [17]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [18]:
len(df.columns)

79

In [19]:
df=df.dropna( axis=0, how='any')
df=df.replace(',,', np.nan, inplace=False)
df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

In [20]:
df.replace("Infinity", 0, inplace=True)
df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)


df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)

df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
df[" CWE Flag Count"].replace(",,", 0, inplace=True)
df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)

df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)

df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)

df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

In [21]:
df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")

In [22]:
df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")

In [23]:
pd.set_option('display.max_rows', df.shape[0])

In [24]:
df.replace('Infinity',0.0, inplace=True)

In [25]:
df.replace('NaN',0.0, inplace=True)

In [26]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [27]:
df_benign=df[df[' Label']=='BENIGN']
df_benign.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0.0,0.0,0.0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0.0,0,0,9.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0.0,0,1,9.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0.0,0,1,9.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0.0,0,1,9.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0.0,0.0,0.0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0.0,0,0,9.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [28]:
df_ddos=df[df[' Label']=='DDoS']
df_ddos.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
18883,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,5840,0,1658.142857,2137.297080,8991.398927,7.729218,1.437547e+05,4.308658e+05,1292730,2,747,373.5,5.239661e+02,744,3,1293746,215624.3333,527671.9348,1292730,2,0,0.0,0.0,0.0,72,152,2.318765,5.410452,0,5840,1057.545455,1853.437529,3435230.673,0,0,0,1,0,0,0.0,0,2,1163.300000,8.666667,1658.142857,0.0,0.0,0.0,0.0,0.0,0.0,3,26,7,11607,8192,229,2,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18884,80,4421382,4,0,24,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,5.428167,0.904695,1.473794e+06,2.552043e+06,4420639,340,4421382,1473794.0,2.552043e+06,4420639,340,0,0.0000,0.0000,0,0,0,0.0,0.0,0.0,80,0,0.904695,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,0,0.0,0,0,7.500000,6.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4,24,0,0,256,-1,3,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18885,80,1083538,3,6,26,11601,20,0,8.666667,10.263203,4380,0,1933.500000,1757.789948,10730.588130,8.306123,1.354422e+05,3.777253e+05,1070206,41,12982,6491.0,8.165669e+03,12265,717,1083407,216681.4000,477167.0977,1070206,41,0,0.0,0.0,0.0,72,132,2.768708,5.537415,0,4380,1162.700000,1645.241762,2706820.456,0,0,0,1,0,0,0.0,0,2,1291.888889,8.666667,1933.500000,0.0,0.0,0.0,0.0,0.0,0.0,3,26,6,11601,8192,229,2,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18886,80,80034360,8,4,56,11601,20,0,7.000000,5.656854,8760,0,2900.250000,4128.319301,145.649943,0.149936,7.275851e+06,2.110000e+07,70600000,6,78700000,11200000.0,2.630000e+07,70600000,247,1354806,451602.0000,777575.2592,1349464,190,0,0.0,0.0,0.0,172,92,0.099957,0.049979,0,8760,897.153846,2488.507044,6192667.308,0,0,0,0,1,0,0.0,0,0,971.916667,7.000000,2900.250000,0.0,0.0,0.0,0.0,0.0,0.0,8,56,4,11601,256,229,6,20,939.0,0.0,939,939,39300000.0,44200000.0,70600000,8072664,DDoS
18887,80,642654,3,6,26,11607,20,0,8.666667,10.263203,5840,0,1934.500000,2538.919278,18101.497850,14.004425,8.033175e+04,2.268247e+05,641694,3,801,400.5,5.409367e+02,783,18,642493,128498.6000,286885.1169,641694,3,0,0.0,0.0,0.0,72,132,4.668142,9.336284,0,5840,1163.300000,2138.329153,4572451.567,0,0,0,1,0,0,0.0,0,2,1292.555556,8.666667,1934.500000,0.0,0.0,0.0,0.0,0.0,0.0,3,26,6,11607,8192,229,2,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS


In [29]:


X=df_benign[df_benign.columns[0:-1]]
y=df_benign[df_benign.columns[-1]]

del df

In [30]:
from scipy import stats


In [31]:


cols = list(X.columns)
for col in cols:
    X[col] = stats.zscore(X[col])
    print(col)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


 Destination Port
 Flow Duration
 Total Fwd Packets
 Total Backward Packets
Total Length of Fwd Packets
 Total Length of Bwd Packets
 Fwd Packet Length Max
 Fwd Packet Length Min
 Fwd Packet Length Mean
 Fwd Packet Length Std
Bwd Packet Length Max
 Bwd Packet Length Min
 Bwd Packet Length Mean
 Bwd Packet Length Std
Flow Bytes/s
 Flow Packets/s
 Flow IAT Mean
 Flow IAT Std
 Flow IAT Max
 Flow IAT Min
Fwd IAT Total
 Fwd IAT Mean
 Fwd IAT Std
 Fwd IAT Max
 Fwd IAT Min
Bwd IAT Total
 Bwd IAT Mean
 Bwd IAT Std
 Bwd IAT Max
 Bwd IAT Min
Fwd PSH Flags


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


 Bwd PSH Flags
 Fwd URG Flags
 Bwd URG Flags
 Fwd Header Length
 Bwd Header Length
Fwd Packets/s
 Bwd Packets/s
 Min Packet Length
 Max Packet Length
 Packet Length Mean
 Packet Length Std
 Packet Length Variance
FIN Flag Count
 SYN Flag Count
 RST Flag Count
 PSH Flag Count
 ACK Flag Count
 URG Flag Count
 CWE Flag Count
 ECE Flag Count
 Down/Up Ratio
 Average Packet Size
 Avg Fwd Segment Size
 Avg Bwd Segment Size
Fwd Avg Bytes/Bulk
 Fwd Avg Packets/Bulk
 Fwd Avg Bulk Rate
 Bwd Avg Bytes/Bulk
 Bwd Avg Packets/Bulk
Bwd Avg Bulk Rate
Subflow Fwd Packets
 Subflow Fwd Bytes
 Subflow Bwd Packets
 Subflow Bwd Bytes
Init_Win_bytes_forward
 Init_Win_bytes_backward
 act_data_pkt_fwd
 min_seg_size_forward
Active Mean
 Active Std
 Active Max
 Active Min
Idle Mean
 Idle Std
 Idle Max
 Idle Min


In [32]:
X.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,1.864799,-0.421474,-0.151613,-0.169201,-0.317394,-0.075181,-0.334656,-0.20243,-0.366483,-0.303627,-0.453575,-0.678765,-0.528766,-0.413824,0.065234,2.803337,-0.316342,-0.353944,-0.364526,-0.048139,-0.397576,-0.242899,-0.302444,-0.328176,-0.099240,-0.404022,-0.228334,-0.332334,-0.340102,-0.106285,-0.252644,NaN,NaN,NaN,-0.163224,-0.173742,3.050954,-0.132489,-0.570327,-0.443463,-0.553631,-0.452023,-0.263324,-0.117474,-0.252644,-0.0146,-0.544403,1.355957,-0.496632,NaN,-0.0146,-0.746670,-0.565224,-0.366483,-0.528766,NaN,NaN,NaN,NaN,NaN,NaN,-0.151613,-0.317394,-0.169201,-0.075181,-0.445265,-0.223151,-0.127972,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
1,1.873217,-0.421471,-0.189113,-0.143173,-0.318971,-0.075095,-0.334656,-0.20243,-0.366483,-0.303627,-0.445898,-0.592415,-0.506577,-0.413824,-0.050959,-0.140004,-0.316306,-0.353944,-0.364520,-0.048016,-0.397576,-0.242899,-0.302444,-0.328177,-0.099241,-0.404022,-0.228334,-0.332334,-0.340102,-0.106285,-0.252644,NaN,NaN,NaN,-0.192789,-0.152327,-0.166232,0.058735,-0.570327,-0.443463,-0.553631,-0.452023,-0.263324,-0.117474,-0.252644,-0.0146,-0.544403,1.355957,2.013564,NaN,-0.0146,-0.024918,-0.565224,-0.366483,-0.506577,NaN,NaN,NaN,NaN,NaN,NaN,-0.189113,-0.318971,-0.143173,-0.075095,-0.445595,-0.192375,-0.176358,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
2,1.873261,-0.421473,-0.189113,-0.143173,-0.318971,-0.075095,-0.334656,-0.20243,-0.366483,-0.303627,-0.445898,-0.592415,-0.506577,-0.413824,-0.047354,-0.048692,-0.316325,-0.353944,-0.364523,-0.048082,-0.397576,-0.242899,-0.302444,-0.328177,-0.099241,-0.404022,-0.228334,-0.332334,-0.340102,-0.106285,-0.252644,NaN,NaN,NaN,-0.192789,-0.152327,-0.117025,0.268346,-0.570327,-0.443463,-0.553631,-0.452023,-0.263324,-0.117474,-0.252644,-0.0146,-0.544403,1.355957,2.013564,NaN,-0.0146,-0.024918,-0.565224,-0.366483,-0.506577,NaN,NaN,NaN,NaN,NaN,NaN,-0.189113,-0.318971,-0.143173,-0.075095,-0.445595,-0.192375,-0.176358,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
3,1.480466,-0.421473,-0.189113,-0.143173,-0.318971,-0.075095,-0.334656,-0.20243,-0.366483,-0.303627,-0.445898,-0.592415,-0.506577,-0.413824,-0.043705,0.043751,-0.316331,-0.353944,-0.364524,-0.048103,-0.397576,-0.242899,-0.302444,-0.328177,-0.099241,-0.404022,-0.228334,-0.332334,-0.340102,-0.106285,-0.252644,NaN,NaN,NaN,-0.192789,-0.152327,-0.067208,0.480554,-0.570327,-0.443463,-0.553631,-0.452023,-0.263324,-0.117474,-0.252644,-0.0146,-0.544403,1.355957,2.013564,NaN,-0.0146,-0.024918,-0.565224,-0.366483,-0.506577,NaN,NaN,NaN,NaN,NaN,NaN,-0.189113,-0.318971,-0.143173,-0.075095,-0.445430,-0.183633,-0.17

In [33]:
# features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets']
# features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,' Flow Duration',  " Total Length of Bwd Packets",  " Flow IAT Mean"," Bwd Packet Length Std",   " Flow IAT Max"]
features=[" Fwd Packet Length Max", " Flow IAT Std"," Fwd Packet Length Std" ,' Flow Duration', " Total Length of Bwd Packets"]

In [34]:
X=X[features].copy()
X.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Flow Duration,Total Length of Bwd Packets
0,-0.334656,-0.353944,-0.303627,-0.421474,-0.075181
1,-0.334656,-0.353944,-0.303627,-0.421471,-0.075095
2,-0.334656,-0.353944,-0.303627,-0.421473,-0.075095
3,-0.334656,-0.353944,-0.303627,-0.421473,-0.075095
4,-0.334656,-0.353944,-0.303627,-0.421474,-0.075181


In [35]:
len(X.columns)

5

In [36]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

In [37]:
import time


In [39]:
t1 = time.time()
from pyod.models.loci import LOCI
clf = LOCI()
clf.fit(X_train[-1024:])

t2 = time.time()


In [40]:
# get the prediction on the test data
y_test_pred = clf.predict(X_test[:300])  # outlier labels (0 or 1)


In [41]:
print("time : ",(t2-t1)/60 )

time :  36.76566710074743


In [42]:
print("Total Samples : ",len(y_test_pred))
print( "Inliers : ",len(y_test_pred[y_test_pred == 0]))
print( "Outliers : ",len(y_test_pred[y_test_pred == 1]))

Total Samples :  300
Inliers :  176
Outliers :  124
